# Dummy plugin smoke test

This notebook runs a tiny forward/loss pass using the `dummy_gpt` model adapter and the
`dummy_text` data module.


In [2]:
from pathlib import Path

import yaml

from llmtrain.config.schemas import RunConfig
from llmtrain.data.dummy_text import DummyTextDataModule
from llmtrain.models.dummy_gpt import DummyGPTAdapter

config_path = Path("../configs/presets/example.yaml")
cfg_dict = yaml.safe_load(config_path.read_text(encoding="utf-8"))
cfg = RunConfig.model_validate(cfg_dict)

cfg = cfg.model_copy(
    update={
        "model": cfg.model.model_copy(
            update={
                "name": "dummy_gpt",
                "vocab_size": 128,
                "d_model": 64,
                "block_size": 32,
            }
        ),
        "data": cfg.data.model_copy(
            update={
                "name": "dummy_text",
                "num_workers": 0,
            }
        ),
        "trainer": cfg.trainer.model_copy(
            update={
                "max_steps": 3,
                "micro_batch_size": 4,
            }
        ),
    }
)

adapter = DummyGPTAdapter()
datamodule = DummyTextDataModule()

model = adapter.build_model(cfg)
tokenizer = adapter.build_tokenizer(cfg)

datamodule.setup(cfg, tokenizer=tokenizer)
loader = datamodule.train_dataloader()
batch = next(iter(loader))

loss, metrics = adapter.compute_loss(model, batch)
print("loss:", float(loss))
print("metrics:", metrics)



loss: 5.107038497924805
metrics: {'loss': 5.107038497924805}


In [5]:
model.parameters

<bound method Module.parameters of _TinyGPT(
  (embed): Embedding(128, 64)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (lm_head): Linear(in_features=64, out_features=128, bias=False)
)>